# KNN (K-Nearest-Neighbors)

KNN is a simple concept: define some distance metric between the items in your dataset, and find the K closest items. You can then use those items to predict some property of a test item, by having them somehow "vote" on it.

As an example, let's look at the MovieLens data. We'll try to guess the rating of a movie by looking at the 10 movies that are closest to it in terms of genres and popularity.

To start, we'll load up every rating in the data set into a Pandas DataFrame:

In [1]:
# %cd /content/drive/MyDrive/colab/MLCourse
import pandas as pd

r_cols = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('./ml-100k/u.data', sep='\t', names=r_cols, usecols=range(3))
ratings.head()


,user_id,movie_id,rating
0,0,50,5
1,0,172,5
2,0,133,1
3,196,242,3
4,186,302,3


Now, we'll group everything by movie ID, and compute the total number of ratings (each movie's popularity) and the average rating for every movie:

In [2]:
import numpy as np

movieProperties = ratings.groupby('movie_id').agg({'rating': [np.size, np.mean]})
movieProperties.head()

rating          
           size      mean
movie_id                 
1           452  3.878319
2           131  3.206107
3            90  3.033333
4           209  3.550239
5            86  3.302326

The raw number of ratings isn't very useful for computing distances between movies, so we'll create a new DataFrame that contains the normalized number of ratings. So, a value of 0 means nobody rated it, and a value of 1 will mean it's the most popular movie there is.

In [3]:
movieNumRatings = pd.DataFrame(movieProperties['rating']['size'])
movieNormalizedNumRatings = movieNumRatings.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
movieNormalizedNumRatings.head()

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798


Now, let's get the genre information from the u.item file. The way this works is there are 19 fields, each corresponding to a specific genre - a value of '0' means it is not in that genre, and '1' means it is in that genre. A movie may have more than one genre associated with it.

While we're at it, we'll put together everything into one big Python dictionary called movieDict. Each entry will contain the movie name, list of genre values, the normalized popularity score, and the average rating for each movie:

In [4]:
movieDict = {}
with open('./ml-100k/u.item') as f:
    temp = ''
    for line in f:
        #line.decode("ISO-8859-1")
        fields = line.rstrip('\n').split('|')
        movieID = int(fields[0])
        name = fields[1]
        genres = fields[5:25]
        genres = map(int, genres)
        movieDict[movieID] = (name, np.array(list(genres)), movieNormalizedNumRatings.loc[movieID].get('size'), movieProperties.loc[movieID].rating.get('mean'))


For example, here's the record we end up with for movie ID 1, "Toy Story":

In [5]:
print(movieDict[232])

('Young Guns (1988)', array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), 0.17152658662092624, 3.207920792079208)


Now let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is. Just to make sure it works, we'll compute the distance between movie ID's 2 and 4:

In [70]:
from scipy import spatial

def ComputeDistance(movie_1, movie_2):
#     genresA = movie_1[1]
#     genresB = movie_2[1]
    genreDistance = spatial.distance.cosine(movie_1[1], movie_2[1])
    popularityA = movie_1[2]
    popularityB = movie_2[2]
    popularityDistance = abs(popularityA - popularityB)
    return (genreDistance**2) + popularityDistance
    

def get_movieID(movie_name: str):
    return next(x for x in movieDict if (movie_name.lower() in movieDict[x][0].lower()))


In [71]:
movie1_id = get_movieID('empire strikes back')
movie2_id = get_movieID('star wars')

print(f'{movieDict[movie1_id]}')
print(f'{movieDict[movie2_id]}')
print(ComputeDistance(movieDict[movie1_id], movieDict[movie2_id]))

('Empire Strikes Back, The (1980)', array([0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]), 0.6295025728987993, 4.206521739130435)
('Star Wars (1977)', array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]), 1.0, 4.359589041095891)
0.3780889020839803


Remember the higher the distance, the less similar the movies are. Let's check what movies 2 and 4 actually are - and confirm they're not really all that similar:

Now, we just need a little code to compute the distance between some given test movie (Toy Story, in this example) and all of the movies in our data set. When the sort those by distance, and print out the K nearest neighbors:

In [86]:
import operator

def getNeighbors(movieID, K):
    distances = []
    for movie in movieDict:
        if (movie != movieID):
            dist = ComputeDistance(movieDict[movieID], movieDict[movie])
            distances.append((movie, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

K = 10
avgRating = 0
movie_name = 'Return of the Jedi'

neighbors = getNeighbors(get_movieID(movie_name), K)
for neighbor in neighbors:
    avgRating += movieDict[neighbor][3]
    print (movieDict[neighbor][0] + " " + str(movieDict[neighbor][3]))
    
avgRating /= K

Star Wars (1977) 4.359589041095891
Independence Day (ID4) (1996) 3.438228438228438
Empire Strikes Back, The (1980) 4.206521739130435
English Patient, The (1996) 3.656964656964657
Raiders of the Lost Ark (1981) 4.252380952380952
Star Trek: First Contact (1996) 3.66027397260274
Princess Bride, The (1987) 4.172839506172839
Indiana Jones and the Last Crusade (1989) 3.930513595166163
Rock, The (1996) 3.693121693121693
Men in Black (1997) 3.745874587458746


While we were at it, we computed the average rating of the 10 nearest neighbors to Toy Story:

In [87]:
avgRating

3.9116308182322554

How does this compare to Toy Story's actual average rating?

In [88]:
movieDict[get_movieID(movie_name)]

('Return of the Jedi (1983)',
 array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0]),
 0.8679245283018868,
 4.007889546351085)

Not too bad!


## Activity

Our choice of 10 for K was arbitrary - what effect do different K values have on the results?

Our distance metric was also somewhat arbitrary - we just took the cosine distance between the genres and added it to the difference between the normalized popularity scores. Can you improve on that?